In [ ]:
from mne.time_frequency import tfr_morlet
from alive_progress import alive_bar
import nibabel as nib
import pandas as pd
import numpy as np
from hm_tools import *
import scipy.io 
import h5py
import tqdm
import mne
import sys
import os

mne_erp_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/data/mne_erp_exp3/'

result_path = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/data/mne_erp_exp3/'



In [ ]:
frequency_band = [[8, 13]]
frequency_band = np.array(frequency_band)
frequency_band_name = ['alpha']
# frequency_band = [[None, 13]]
# frequency_band = np.array(frequency_band)
# frequency_band_name = ['delta_theta_alpha']
# 去除数据残缺的被试
bad_subject = np.array([3, 5, 15]) -1
all_subject = np.arange(0,34,1)

# 删掉特定元素
good_subject = np.setdiff1d(all_subject, bad_subject)

In [ ]:

for iter_subject in good_subject:

    # 生成每个被试的结果存放地址
    subject_file = result_path + '/subject_' + str(iter_subject + 1)

    # 检查地址是否存在
    if not os.path.exists(subject_file):
        os.makedirs(subject_file)

    # read the epoch data
    eeg_epochs = mne.read_epochs(subject_file + '/ERP_epo.fif', preload = True)
    for iter_frequency in range(len(frequency_band_name)):
        filtered_eeg=eeg_epochs.copy().filter(l_freq=frequency_band[iter_frequency,0], 
                                            h_freq=frequency_band[iter_frequency,1], verbose=None)

        # save the mne data
        filtered_eeg.save(subject_file +'/' + frequency_band_name[iter_frequency] + '_epo.fif', overwrite=True)

    print('')
    print("**************************************************************************") 
    print("******************** subject number:", iter_subject + 1, "/", 34,"finished *********************")
    print("**************************************************************************")        
    print('')

In [ ]:

# 时间窗的长度， 单位
time_window = 0.02  

# 需要分析的数据起止时间 第一位是开始的时间，第二位是结束的时间
involve_time = [-0.200, 1.000] 


In [ ]:
mne_erp_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/data/mne_erp_exp1/'

result_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/result/varibilit_corr_20_exp2_new_all_step_corr/'

# power band name
file_name = ['alpha']

# the number of all the subjects
subject_num = 34

# the length of time window (s)
time_window = 0.02  

# Start and end time of the data to be analyzed The first time is the start time and the second time is the end time  
involve_time = [-0.200, 1.000] 

In [ ]:

def varibility_corr(eeg_epochs, involve_time, window_length):
    # The time series of involve_time and epoch were compared to find out which digit started to calculate the correlation  
    time_series = np.array(eeg_epochs.times)
    start_time = np.where(time_series == involve_time[0])[0][0]
    end_time = np.where(time_series == involve_time[1])[0][0]

    # get eeg data from mne file
    eeg_data = eeg_epochs.get_data()# trail * channel * time

    _, not_nan_position = find_1d_array_nan(eeg_data[:,0,0])
    eeg_data = eeg_data[not_nan_position]

    # Generates an empty total result array
    iter_subject_result = []

    # Cycle each electrode once
    for iter_elec in tqdm(range(eeg_data.shape[1])):

        # Generates an empty result array for each electrode
        iter_elec_result = []
        # Calculate the variability once for each time window
        for iter_window in range(end_time - start_time):# 步长为1
            # Get the data for each time window
            iter_data = eeg_data[:,iter_elec,(start_time + iter_window - round(window_length/2)) : (start_time + iter_window + round(window_length/2))]
            # The correlation coefficients are calculated for the matrix 
            corr = np.corrcoef(iter_data)
            # Get the lower triangle below the diagonal (k=-1)
            low_triangle = np.tril(corr, k=-1)
            # Sum of the lower trig
            all_corr = np.sum(low_triangle)
            # 计算下三角的数据点个数
            corr_num = (eeg_data.shape[0]**2 - eeg_data.shape[0])/2
            # Calculate the number of data points in the lower triangle
            average_corr = all_corr/corr_num
            # Calculate the variation by 1-corr
            average_var = 1 - average_corr
            # Store results to results for each electrode
            iter_elec_result.append(average_var)
        # Store results to the total result matrix
        iter_subject_result.append(list(iter_elec_result))
    return iter_subject_result #Output result matrix  (elec * time)

In [ ]:
for iter_subject in good_subject:

    # Generate the eeg data storage address of each subject
    subject_erp_loc = mne_erp_loc + '/subject_' + str(iter_subject + 1)

    # Generate the storage address of the variation results of each subject
    subject_result_loc = result_loc + '/subject_' + str(iter_subject + 1)
    if not os.path.exists(subject_result_loc):
        os.makedirs(subject_result_loc)
    
    # Alpha band only
    for iter_file in range(len(file_name)):
        
        # read the epoch data
        eeg_epochs = mne.read_epochs(subject_erp_loc +'/' + file_name[iter_file] + '_epo.fif', preload = True)

        # Check the number of event
        event_name = eeg_epochs.event_id.keys()
        event_name = list(event_name)

        # create the dict for save the result
        var_result = {}

        # input some basic information in the dict result
        var_result['ch_names'] = eeg_epochs.ch_names
        var_result['fs'] = eeg_epochs.info['sfreq']
        
        window_length = eeg_epochs.info['sfreq'] * time_window 
        print('point of the window is:', window_length)

        var_erp_all = varibility_corr(eeg_epochs, involve_time, window_length)

        # save the all_event result to the dict
        var_result['all_event'] = var_erp_all    

        # Calculate variation for each event
        for iter_event in range(len(event_name)):

            var_erp_iter_event = varibility_corr(eeg_epochs[event_name[iter_event]], involve_time, window_length)

            # save the result per event
            var_result[event_name[iter_event]] = var_erp_iter_event


        # save the mne data
        np.save(subject_result_loc +'/' + file_name[iter_file] + '_corr.npy', var_result)

    print('')
    print("**************************************************************************") 
    print("******************** subject number:", iter_subject + 1, "/", subject_num,"finished *********************")
    print("**************************************************************************")        
    print('')

In [ ]:
var_loc = 'E:/workspace/Trust_Data_and_Results/Haoming/varibility/result/varibilit_corr_20_exp2_new_all_step_corr/'

# Extract the data from each participant’s folder and integrate them into a large matrix
all_data = []
for iter_subject in good_subject:

    # Generate the erp storage address of each subject
    subject_erp_loc = var_loc + '/subject_' + str(iter_subject + 1)

    iter_subject_data = []
    for iter_file in range(len(file_name)):

        file_loc = subject_erp_loc + '/' + file_name[iter_file] + '_corr.npy'

        var_result = np.load(file_loc, allow_pickle=True).item()

        iter_file_data = []
        for inter_event in range(len(event_name)):

            iter_data = var_result[event_name[inter_event]]
            iter_file_data.append(list(iter_data))
        
        iter_subject_data.append(iter_file_data)
    #iter_subject_data.shape
    all_data.append(iter_subject_data)

all_data = np.array(all_data)
all_data.shape
            

all_var_result = {}
all_var_result['shape'] = ['subjects', 'power_band', 'event_name', 'ch_names', 'time_point']
all_var_result['power_band'] = file_name
all_var_result['event_name'] = event_name
all_var_result['ch_names'] = var_result['ch_names']
all_var_result['fs'] = var_result['fs']
all_var_result['data'] = all_data
np.save(result_loc + '/all_var_data.npy', all_var_result)

# normalize all the data by divide the by -200 ~ 0s
all_data_nor = list([])
for iter_subject in range(31):

    # 生成每个被试的erp存放地址
    subject_erp_loc = var_loc + '/subject_' + str(iter_subject + 1)

    iter_subject_data_nor = []
    for iter_file in range(len(file_name)):

        iter_file_data_nor = []
        for iter_event in range(len(event_name)):
            
            iter_elec_data_nor=[]
            for iter_elec in range(63):

                var_data = np.squeeze(all_data[iter_subject, iter_file, iter_event, iter_elec])
                var_data_nor = var_data / np.mean(var_data[0:200])
                iter_elec_data_nor.append(list(var_data_nor))

            iter_file_data_nor.append(list(iter_elec_data_nor))
        
        iter_subject_data_nor.append(iter_file_data_nor)
    #iter_subject_data.shape
    all_data_nor.append(iter_subject_data_nor)

all_data_nor = np.array(all_data_nor)
all_data_nor.shape


all_var_result_nor = {}
all_var_result_nor['shape'] = ['subjects', 'power_band', 'event_name', 'ch_names', 'time_point']
all_var_result_nor['power_band'] = file_name
all_var_result_nor['event_name'] = event_name
all_var_result_nor['ch_names'] = var_result['ch_names']
all_var_result_nor['fs'] = var_result['fs']
all_var_result_nor['data'] = all_data_nor
all_var_result_nor['tmin'] = -0.2
all_data_nor = all_var_result_nor
np.save(result_loc + '/all_var_data_nor.npy', all_var_result_nor)

In [ ]:
# 这个只计算三种情况相对于0的test
permu_result_loc = result_loc + '/permutation_result/'
n_permutations=10000
permutation_cluster_result = {}
# calculate std

tfr_ROI_epoch_data_std={}
tfr_ROI_epoch_data_std['all_event_std'] = np.std(np.squeeze(all_data_nor['data'][:, :, 0, :, :]), axis=0)
tfr_ROI_epoch_data_std['event1_std'] = np.std(np.squeeze(all_data_nor['data'][:, :, 1, :, :]), axis=0)
tfr_ROI_epoch_data_std['event2_std'] = np.std(np.squeeze(all_data_nor['data'][:, :, 2, :, :]), axis=0)

permutation_cluster_result['std_error'] = tfr_ROI_epoch_data_std

print('')
print("**************************************************************************") 
#print("*"*10)
print("************************ std calculation finished ************************")
print("**************************************************************************")        
print('')

      
for ROI_num in range(63):
    #ROI_num = 10
    # compute the cluster test for event 1
    T_obs, clusters, cluster_p_values, H0  = mne.stats.permutation_cluster_1samp_test(all_data_nor['data'][:, 3, 1, ROI_num, :]-1, out_type='mask',n_permutations=n_permutations, tail=0, verbose=None)
    event1_result = {'T_obs':T_obs, 'clusters':clusters, 'cluster_p_values':cluster_p_values, 'H0':H0}

    # compute the cluster test for event 2
    T_obs, clusters, cluster_p_values, H0  = mne.stats.permutation_cluster_1samp_test(all_data_nor['data'][:, 3, 2, ROI_num, :]-1, out_type='mask',n_permutations=n_permutations, tail=0, verbose=None)
    event2_result = {'T_obs':T_obs, 'clusters':clusters, 'cluster_p_values':cluster_p_values, 'H0':H0}

    # compute the cluster test for  all event 
    T_obs, clusters, cluster_p_values, H0  = mne.stats.permutation_cluster_1samp_test(all_data_nor['data'][:, 3, 0, ROI_num, :]-1, out_type='mask',n_permutations=n_permutations, tail=0, verbose=None)
    all_event_result = {'T_obs':T_obs, 'clusters':clusters, 'cluster_p_values':cluster_p_values, 'H0':H0}

    # put them in a dict
    permutation_cluster_result_per_roi = {'event1_result':event1_result, 'event2_result':event2_result, 'all_event_result':all_event_result}
    permutation_cluster_result[ROI_num] = permutation_cluster_result_per_roi


    print('')
    print("**************************************************************************")
    print("********************* total number:", ROI_num + 1, "/", 63,"finished ***********************")
    print("**************************************************************************")
    print('')

if not os.path.exists(permu_result_loc):
    os.makedirs(permu_result_loc)
np.save(permu_result_loc + 'all_event_0_1_alpha' + '.npy', permutation_cluster_result)




In [ ]:
# plot function
import matplotlib.pyplot as plt
import numpy as np
import os
def plot_erp(permutation_cluster_result, epoch_mean, epoch_data_std, times, event_name, line_color=['orangered','limegreen'], 
            figsize=(14,6), title_size=20, legend_size=15, labelsize=15, ticksize=20, subplots_adjust=[0.15, 0.15, 0.85, 0.85]):
    # keys of the dict permutation_cluster_result
    #'event1_result':event1_result, 'event2_result':event2_result, 'compare_result':compare_result}
    #'T_obs':T_obs, 'clusters':clusters, 'cluster_p_values':cluster_p_values, 'H0':H0

    # keys of the dict epoch_data_mean
    # 'event_0' (34, 1300)
    # plot for each ROI

    event_0_line_color = line_color[0]
    event_1_line_color = line_color[1]
    event_2_line_color = line_color[2]


    plt.close('all')
    plt.rcParams['figure.figsize'] = figsize # 设置figure_size尺寸

    plt.plot(times, epoch_mean['all_event'], color=event_0_line_color, linestyle='--', alpha=0.4)
    plt.plot(times, epoch_mean['event_0'], color=event_1_line_color, linestyle='--', alpha=0.4)
    plt.plot(times, epoch_mean['event_1'], color=event_2_line_color, linestyle='--', alpha=0.4)


    # Plot std
    std_event0 = epoch_data_std['all_event']
    std_event1 = epoch_data_std['event_0']
    std_all_event = epoch_data_std['event_1']

    plt.fill_between(times, epoch_mean['all_event'] - std_all_event, epoch_mean['all_event'] + std_all_event, color=event_0_line_color, alpha=0.1)
    plt.fill_between(times, epoch_mean['event_0'] - std_event0, epoch_mean['event_0'] + std_event0, color=event_1_line_color, alpha=0.1)
    plt.fill_between(times, epoch_mean['event_1'] - std_event1, epoch_mean['event_1'] + std_event1, color=event_2_line_color, alpha=0.1)

    # Plot the significant of event 0 
    for i_c, c in enumerate(permutation_cluster_result['all_event_result']['clusters']):
        c = c[0]
        if permutation_cluster_result['all_event_result']['cluster_p_values'][i_c] <= 0.05:
            plt.plot(times[c.start : c.stop - 1], epoch_mean['all_event'][c.start : c.stop-1], color=event_0_line_color, alpha=0.9)

    # Plot the significant of event 1 
    for i_c, c in enumerate(permutation_cluster_result['event1_result']['clusters']):
        c = c[0]
        if permutation_cluster_result['event1_result']['cluster_p_values'][i_c] <= 0.05:
            plt.plot(times[c.start : c.stop - 1], epoch_mean['event_0'][c.start : c.stop-1], color=event_1_line_color, alpha=0.9)

    # Plot the significant between event 1 and 2   
    for i_c, c in enumerate(permutation_cluster_result['event2_result']['clusters']):
        c = c[0]
        if permutation_cluster_result['event2_result']['cluster_p_values'][i_c] <= 0.05:
            plt.plot(times[c.start : c.stop - 1], epoch_mean['event_1'][c.start : c.stop-1], color=event_2_line_color, alpha=0.9)

    plt.subplots_adjust(left=subplots_adjust[0], bottom=subplots_adjust[1], right=subplots_adjust[2], top=subplots_adjust[3],                         hspace=0.1,wspace=0.1)

    plt.xlim([times[0]-0.02, times[-1]+0.02])

    plt.yticks(size=ticksize, family='Arial')
    plt.xticks(size=ticksize, family='Arial')


    return plt


In [ ]:
# plot varibility erps
plot_result_loc = result_loc + '/alpha_plot_result/'
permutation_cluster_result = np.load(permu_result_loc + 'all_event_0_1_alpha' + '.npy', allow_pickle=True).item()


figsize=(10,6)
title_size = 20
labelsize = 15
ticksize=25
# all_data_nor = np.load(var_loc + '/all_var_data_nor_3band.npy', allow_pickle=True).item()

# for iter_file in range(len(file_name)):


if not os.path.exists(plot_result_loc):
    os.makedirs(plot_result_loc)

# iter_permutation_cluster_result = np.load(permu_result_loc + file_name[iter_file] + '.npy', allow_pickle=True).item()
for iter_elec in range(63):

    epoch_data_std = {}
    epoch_data_std['all_event'] = np.std(np.squeeze(all_data_nor['data'][:, 3, 0, iter_elec, :]), axis=0)
    epoch_data_std['event_0'] = np.std(np.squeeze(all_data_nor['data'][:, 3, 1, iter_elec, :]), axis=0)
    epoch_data_std['event_1'] = np.std(np.squeeze(all_data_nor['data'][:, 3, 2, iter_elec, :]), axis=0)

    epoch_data_mean = {}
    epoch_data_mean['all_event'] = np.average(np.squeeze(all_data_nor['data'][:, 3, 0, iter_elec, :]), axis=0)
    epoch_data_mean['event_0'] = np.average(np.squeeze(all_data_nor['data'][:, 3, 1, iter_elec, :]), axis=0)
    epoch_data_mean['event_1'] = np.average(np.squeeze(all_data_nor['data'][:, 3, 2, iter_elec, :]), axis=0)

    times = np.arange(-0.2, 1, 0.001)
    event_name = ['Trust & Untrust', 'Trust', 'Untrust']


    plt = plot_erp(permutation_cluster_result[iter_elec], epoch_data_mean, epoch_data_std, 
    times, event_name, line_color=['#DF4058','#70AD47','#0070C0'],
    ticksize = ticksize, figsize= figsize)

    plt.title(all_data_nor['ch_names'][iter_elec], fontdict= {'fontsize':title_size})

    plt.xlabel("time (s)", fontsize=labelsize)
    plt.ylabel("varibility change", fontsize=labelsize)

    # plot the cross line
    plt.axvline(times[201], c="gray", ls = "dashed")
    plt.plot(times, np.ones(len(times)), color="gray", linestyle="dashed")

    plt.savefig(plot_result_loc + '/' + all_data_nor['ch_names'][iter_elec] + ".png")

